# Combine two texts changing words to quasisinonyms

In [1]:
import os
import re
import gensim
import pymorphy2

c:\users\tatiana\appdata\local\programs\python\python35-32\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
directory = ## YOUR WORKING DIRECTIRY HERE ##

# Document loader

Load sample files in docs format

In [3]:
import docx

def getText(filename1, filename2):
    doc1 = docx.Document(filename1)
    doc2 = docx.Document(filename2)
    para1 = doc1.paragraphs
    para2 = doc2.paragraphs
    para1 = [p for p in para1 if p.text!='']
    para2 = [p for p in para2 if p.text!='']
    fullText = []
    j = 0
    k = 0
    for i in range(len(para1) + len(para2)):
        if i % 2 == 0:
            try:
                fullText.append(para1[j].text)
            except IndexError:
                continue
            j += 1
        else:
            try:
                fullText.append(para2[k].text)
            except IndexError:
                continue
            k += 1
    return '\n'.join(fullText)

# Text transformer

In [4]:
morph = pymorphy2.MorphAnalyzer()
punct = re.compile('^(.*?)([а-яА-ЯёЁ-]+)(.*?)$')

In [5]:
capit_letters = [chr(x) for x in range(1040,1072)] + ['Ё']

In [6]:
def search_neighbour(word, pos, gend='masc'):
    word = word.replace('ё', 'е')
    lex = word + '_' + cotags[pos]
    if lex in model:
        neighbs = model.most_similar([lex], topn=20)
        for nei in neighbs:
            lex_n, ps_n = nei[0].split('_')
            if '::' in lex_n:
                continue
            if cotags[pos] == ps_n:
                if pos == 'NOUN':
                    parse_result = morph.parse(lex_n)
                    for ana in parse_result:
                        if ana.normal_form == lex_n:
                            if ana.tag.gender == gend:
                                return lex_n
                elif cotags[pos] == 'VERB' and word[-2:] == 'ся':
                    if lex_n[-2:] == 'ся':
                        return lex_n
                elif cotags[pos] == 'VERB' and word[-2:] != 'ся':
                    if lex_n[-2:] != 'ся':
                        return lex_n
                else:
                    return lex_n
    return None

In [7]:
def flection(lex_neighb, tags):
    tags = str(tags)
    tags = re.sub(',[AGQSPMa-z-]+? ', ',', tags)
    tags = tags.replace("impf,", "")
    tags = re.sub('([A-Z]) (plur|masc|femn|neut|inan)', '\\1,\\2', tags)
    tags = tags.replace("Impe neut", "")
    tags = tags.split(',')
    tags_clean = []
    for t in tags:
        if t:
            if ' ' in t:
                t1, t2 = t.split(' ')
                t = t2
            tags_clean.append(t)
    tags = frozenset(tags_clean)
    prep_for_gen = morph.parse(lex_neighb)
    ana_array = []
    for ana in prep_for_gen:
        if ana.normal_form == lex_neighb:
            ana_array.append(ana)
    for ana in ana_array:
        try:
            flect = ana.inflect(tags)
        except:
            print(tags)
            return None
        if flect:
            word_to_replace = flect.word
            return word_to_replace
    return None


In [8]:
# начинается ли слово с большой буквы?
capit = re.compile('^[А-Я]+$')
def capit_flagging(wordform):
    if wordform[0] in capit_letters: 
                        return 1
    return 0

In [24]:
# ищем ближайшие слова в модели, для существительных учитываем род
def find_neighb(pos_tag, parse_results, lex, parse_result):
    if pos_tag == 'NOUN':
        gen_tag = parse_result.tag.gender
        lex_neighb = search_neighbour(lex, pos_tag, gend=gen_tag)
    else:
        lex_neighb = search_neighbour(lex, pos_tag)
    return lex_neighb

In [25]:
# ищем соседа и кэшируем, если он еще не кэширован
def find_if_not_cashed(lex, pos_tag, cash_neighb, parse_results):
    if (lex, pos_tag) in cash_neighb:
        lex_neighb = cash_neighb[(lex, pos_tag)]
    else:
        lex_neighb = find_neighb(pos_tag, parse_results, lex, parse_results) # ищем ближайший квазисиноним
        cash_neighb[(lex, pos_tag)] = lex_neighb # запомнили его на будущее
    return lex_neighb, cash_neighb

In [27]:
# put noun in the right form
def nouns(parse_result, capit_flag, lex_neighb, struct, word):
    if parse_result.tag.case == 'nomn' and parse_result.tag.number == 'sing':
        if capit_flag == 1:
            lex_neighb = lex_neighb.capitalize()
        return struct[0] + lex_neighb + struct[2]
    else:
        word_to_replace = flection(lex_neighb, parse_result.tag)
        if word_to_replace:
            if capit_flag == 1:
                word_to_replace = word_to_replace.capitalize()
            return struct[0] + word_to_replace + struct[2]
        else:
            return word

In [38]:
# put adjectives in the right form
def adjectives(parse_result, capit_flag, lex_neighb, struct, word):
    if parse_result.tag.case == 'nomn' and parse_result.tag.number == 'sing' and parse_result.tag.gender == 'masc': 
        if capit_flag == 1:
            lex_neighb = lex_neighb.capitalize()
        return struct[0] + lex_neighb + struct[2]
    else:
        word_to_replace = flection(lex_neighb, parse_result.tag)
        if word_to_replace:
            if capit_flag == 1:
                word_to_replace = word_to_replace.capitalize()
            return struct[0] + word_to_replace + struct[2]
        else:
            return word

In [29]:
# put a word in the right form
def put_in_the_right_form(pos_tag, parse_result, capit_flag, lex_neighb, struct, word):
    if pos_tag == 'NOUN':
        return nouns(parse_result, capit_flag, lex_neighb, struct, word)                         
    elif pos_tag == 'ADJF':
        return adjectives(parse_result, capit_flag, lex_neighb, struct, word)                        
    elif pos_tag == 'INFN':
        if capit_flag == 1:
            lex_neighb = lex_neighb.capitalize()
        return struct[0] + lex_neighb + struct[2]                         
    elif pos_tag in ['ADVB', 'COMP', 'PRED']:
        if capit_flag == 1:
            lex_neighb = lex_neighb.capitalize()
        return struct[0] + lex_neighb + struct[2]    
    else:
        word_to_replace = flection(lex_neighb, parse_result.tag)
        if word_to_replace:
            if capit_flag == 1:
                word_to_replace = word_to_replace.capitalize()
            return struct[0] + word_to_replace + struct[2]
        else:
            return word

In [34]:
def transform_text(line):
    cash_neighb = {}
    new_line = []
    line = text_example.strip()
    words = line.split(' ')
    for word in words:
        struct = punct.findall(word)
        if struct:
            struct = struct[0]
        else:
            new_line.append(word)
            continue
        wordform = struct[1]
        if wordform:
            if capit.search(wordform): # аббревиатуры (все буквы большие) не меняем
                new_line.append(word)
                continue            
            capit_flag = capit_flagging(wordform) # начинается ли слово с большой буквы?
            parse_result = morph.parse(wordform)[0]
            
            if 'Name' in parse_result.tag or 'Patr' in parse_result.tag: # Не меняем личные имена и названия мест
                    new_line.append(word)
                    continue
                        
            pos_flag = 0
            for tg in cotags:
                if tg in parse_result.tag:
                    pos_flag = 1
                    lex = parse_result.normal_form
                    pos_tag = parse_result.tag.POS
                    lex_neighb, cash_neighb = find_if_not_cashed(lex, pos_tag, cash_neighb, parse_result)
                    if not lex_neighb: # если к слову нет синонимов, не меняем его
                        new_line.append(word)
                        break
                    else: # Иначе нам нужно поставить его в правильную форму
                        to_append = put_in_the_right_form(pos_tag, parse_result, capit_flag, lex_neighb, struct, word)
                        new_line.append(to_append)
                        break
            if pos_flag == 0:
                new_line.append(word)
        else:
            new_line.append(''.join(struct))
        line_replace = ' '.join(new_line)
    return line_replace

# Load pretrained word2vec model and transform text

# rusvectores model trained on 600 millions words (wiki+ruscorpora)

In [16]:
path = directory + '/models/ruwikiruscorpora_0_300_20.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
model.init_sims(replace=True)

check if the model is working properly

In [17]:
model.most_similar(['год_NOUN'], topn=20)

[('гг_NOUN', 0.6715219020843506),
 ('официально_ADV', 0.5154324769973755),
 ('месяц_NOUN', 0.4920658469200134),
 ('ранее_ADV', 0.4650496542453766),
 ('впоследствии_ADV', 0.46287965774536133),
 ('период_NOUN', 0.4466725289821625),
 ('вскоре_ADV', 0.428223192691803),
 ('великобритания_NOUN', 0.4216790795326233),
 ('руководство_NOUN', 0.4185072183609009),
 ('незадолго_ADV', 0.41545569896698),
 ('поздно_ADV', 0.4105498194694519),
 ('совместно_ADV', 0.40229037404060364),
 ('первоначально_ADV', 0.40022385120391846),
 ('го_NOUN', 0.3999163508415222),
 ('полгода_NOUN', 0.3921680450439453),
 ('санкт-петербург_NOUN', 0.3864089250564575),
 ('ежегодно_ADV', 0.3858228325843811),
 ('нидерланды_NOUN', 0.3828669786453247),
 ('ганновер_NOUN', 0.3802339434623718),
 ('фактически_ADV', 0.3796970546245575)]

pretrained model has other morphological tags, need to transform them

In [18]:
cotags = {'ADJF':'ADJ', # pymorphy2: word2vec 
'ADJS' : 'ADJ', 
'ADVB' : 'ADV', 
'COMP' : 'ADV', 
'GRND' : 'VERB', 
'INFN' : 'VERB', 
'NOUN' : 'NOUN', 
'PRED' : 'ADV', 
'PRTF' : 'ADJ', 
'PRTS' : 'VERB', 
'VERB' : 'VERB'}

Load texts and transform

In [45]:
docnames = ['Педагогика', 'Кадровое делопроизводство', 'Строительство', 'Транспорт', 'Маркшейдерия']


In [46]:
for docname in docnames:
    path1 = directory + '/SampleTexts' + '/' + docname +'1.docx'
    path2 = directory + '/SampleTexts' + '/' + docname +'2.docx'

    text_example = getText(path1, path2)
    new_text = transform_text(text_example)
    path = directory + '/results/'
    if not os.path.exists(path):
        os.makedirs(path)

    document = docx.Document()
    p = document.add_paragraph(new_text)
    document.save(path + 'wiki_' + docname + '.docx')

# our model trained on ~ 600 thousands of words

In [47]:
cotags = {'ADJF':'ADJF', # pymorphy2: pymorphy2 
'ADJS' : 'ADJS', 
'ADVB' : 'ADVB', 
'COMP' : 'COMP', 
'GRND' : 'GRND', 
'INFN' : 'INFN', 
'NOUN' : 'NOUN', 
'PRED' : 'PRED', 
'PRTF' : 'PRTF', 
'PRTS' : 'PRTS', 
'VERB' : 'VERB'}

In [53]:
path = directory + '/models/mymodel.bin'
model = gensim.models.Word2Vec.load(path)
model.init_sims(replace=True)

check if the model is working properly

In [54]:
model.most_similar(['год_NOUN'], topn=20)

[('пять_NUMR', 0.8945003747940063),
 ('раз_NOUN', 0.8592861890792847),
 ('реж_NOUN', 0.8405357599258423),
 ('согласно_PREP', 0.7899616360664368),
 ('начальник_NOUN', 0.7609720230102539),
 ('должность_NOUN', 0.7586870789527893),
 ('руководящий_ADJF', 0.749430775642395),
 ('рядовой_ADJF', 0.7473839521408081),
 ('утвердить_PRTS', 0.7447465658187866),
 ('появиться_VERB', 0.7317184209823608),
 ('трудоустройство_NOUN', 0.7218949794769287),
 ('устраиваться_INFN', 0.7142784595489502),
 ('законодательно_ADVB', 0.7088960409164429),
 ('прервать_VERB', 0.7014658451080322),
 ('руководитель_NOUN', 0.6981256008148193),
 ('переходить_INFN', 0.6969155073165894),
 ('врач-гастроэнтеролог_NOUN', 0.6964960694313049),
 ('проходить_INFN', 0.6951399445533752),
 ('бухгалтер_NOUN', 0.6890097856521606),
 ('претендовать_INFN', 0.6848532557487488)]

Load texts and transform

In [51]:
docnames = ['Педагогика', 'Кадровое делопроизводство', 'Строительство', 'Транспорт', 'Маркшейдерия']

In [55]:
for docname in docnames:
    path1 = directory + '/SampleTexts' + '/' + docname +'1.docx'
    path2 = directory + '/SampleTexts' + '/' + docname +'2.docx'

    text_example = getText(path1, path2)
    new_text = transform_text(text_example)
    path = directory + '/results/'
    if not os.path.exists(path):
        os.makedirs(path)

    document = docx.Document()
    p = document.add_paragraph(new_text)
    document.save(path + 'mymodel_' + docname + '.docx')